# Extract frames from a video

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf


with initialize(version_base=None, config_path="conf", ):
    cfg = compose(config_name="config.yaml")

import buttermilk
bm = buttermilk.BM(cfg=cfg)


2024-08-26 23:11:12 26f087537f3a buttermilk buttermilk.py[ 202] INFO Logging setup for: {'function_name': 'fyp', 'job': 'test', 'logs': '20240826T1311Z-ipVh-26f087537f3a-vscode', 'user': 'vscode', 'node': '26f087537f3a'}. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'fyp', 'job': 'test', 'task_id': '20240826T1311Z-ipVh-26f087537f3a-vscode'})). Default save directory for data in this run is: gs://dmrc-analysis/runs/fyp/test/20240826T1311Z-ipVh-26f087537f3a-vscode
Prompt flow service has started...


You can view the trace detail from the following URL:
http://127.0.0.1:23334/v1.0/ui/traces/?#collection=fyp&uiTraceId=0x83cde4b9569242c984e41b78edb3afb9
https://ai.azure.com/projecttrace/detail/0x83cde4b9569242c984e41b78edb3afb9?wsid=/subscriptions/7e7e056a-4224-4e26-99d2-1e3f9a688c50/resourceGroups/rg-suzor_ai/providers/Microsoft.MachineLearningServices/workspaces/automod
You can view the trace detail from the following URL:
http://127.0.0.1:23334/v1.0/ui/traces/?#collection=fyp&uiTraceId=0x15b085a930630ba318e710d42219145f
https://ai.azure.com/projecttrace/detail/0x15b085a930630ba318e710d42219145f?wsid=/subscriptions/7e7e056a-4224-4e26-99d2-1e3f9a688c50/resourceGroups/rg-suzor_ai/providers/Microsoft.MachineLearningServices/workspaces/automod


In [3]:

print(OmegaConf.to_yaml(cfg))


name: fyp
job: test
project:
  secret_provider: azure
  logger: gcp
  models_secret: models
  save_dest: gcp
  save_dir: null
  gcp:
    project: dmrc-analysis
    region: us-central1
    bucket: dmrc-analysis
  azure:
    vault: https://suzorvault.vault.azure.net/
    resource_group: rg-suzor_ai
  aws:
    secret_name: secret
    region: us-east-1
run:
  dataset:
    uri: gs://dmrc-analysis/test/fyp/*
  frame:
    prompt_template_path: generic.prompty
    system_prompt: system_frames.jinja2
    output_format: json_frames.jinja2
    instructions: instructions_frames.jinja2
    langchain_model_name: gemini15pro



In [4]:
import pandas as pd
from buttermilk.flows.extract import Analyst

results = pd.DataFrame()
flow_vars = cfg['run']['frame']
flow = Analyst(**flow_vars)

/opt/conda/envs/bm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-08-26 23:11:21 26f087537f3a buttermilk buttermilk.py[ 202] INFO Logging setup for: {'function_name': 'default_project', 'job': 'development', 'logs': '20240826T1311Z-WfPA-26f087537f3a-vscode', 'user': 'vscode', 'node': '26f087537f3a'}. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'default_project', 'job': 'development', 'task_id': '20240826T1311Z-WfPA-26f087537f3a-vscode'})). Default save directory for data in this run is: gs://dmrc-analysis/runs/default_project/development/20240826T1311Z-WfPA-26f087537f3a-vscode


INFO:buttermilk:Logging setup for: {'function_name': 'default_project', 'job': 'development', 'logs': '20240826T1311Z-WfPA-26f087537f3a-vscode', 'user': 'vscode', 'node': '26f087537f3a'}. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'default_project', 'job': 'development', 'task_id': '20240826T1311Z-WfPA-26f087537f3a-vscode'})). Default save directory for data in this run is: gs://dmrc-analysis/runs/default_project/development/20240826T1311Z-WfPA-26f087537f3a-vscode


Prompt flow service has started...


In [6]:
response = flow(content='', media_attachment_uri='gs://dmrc-analysis/test/fyp/tiktok-imane-01.mp4', model="gemini15pro")

import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(response)

KeyboardInterrupt: 

In [10]:

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, SystemMessage

from langchain_core.prompts import ChatMessagePromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
msg = HumanMessage(
    content=[{
            "type": 'text',
            "text": "describe this image or video",
        },{
            "type": 'image_url',
            "image_url": "gs://dmrc-analysis/test/fyp/tiktok-imane-01.mp4"
        },
    ])

tpl  = ChatPromptTemplate(messages=[msg])

from buttermilk.tools.json_parser import ChatParser
from buttermilk import BM
from buttermilk.llms import LLMs
bm = BM()
connections = bm._connections_azure
llm = LLMs(connections=connections)["gemini15pro"]
chain = tpl | llm | ChatParser()
result = chain.invoke(input={})
#{'file_data': {'mime_type': 'video/mp4',
 # 'file_uri': 'gs://dmrc-platforms/test/fyp/tiktok-imane-02.mp4'}}

/opt/conda/envs/bm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-08-27 14:56:04 26f087537f3a buttermilk buttermilk.py[ 199] INFO Logging setup for: {'function_name': 'default_project', 'job': 'development', 'logs': None, 'user': 'vscode', 'node': '26f087537f3a'}. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'default_project', 'job': 'development', 'task_id': None})). Default save directory for data in this run is: gs://dmrc-analysis/runs/default_project/development/None


Failed to submit 1 logs.
Traceback (most recent call last):
  File "/opt/conda/envs/bm/lib/python3.11/site-packages/google/protobuf/json_format.py", line 565, in _ConvertFieldValuePair
    self._ConvertMapFieldValue(value, message, field,
  File "/opt/conda/envs/bm/lib/python3.11/site-packages/google/protobuf/json_format.py", line 741, in _ConvertMapFieldValue
    getattr(message, field.name)[key_value] = _ConvertScalarFieldValue(
                                              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/bm/lib/python3.11/site-packages/google/protobuf/json_format.py", line 779, in _ConvertScalarFieldValue
    if _UNPAIRED_SURROGATE_PATTERN.search(value):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: expected string or bytes-like object, got 'NoneType'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/envs/bm/lib/python3.11/site-packages/google/protobuf/json_format.py", line 600

Prompt flow service has started...


TypeError: string indices must be integers, not 'str'

In [1]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting, FinishReason
import vertexai.generative_models as generative_models
msg=Part.from_uri(
    mime_type="video/mp4",
    uri="gs://dmrc-platforms/test/fyp/tiktok-imane-02.mp4")

msg

file_data {
  mime_type: "video/mp4"
  file_uri: "gs://dmrc-platforms/test/fyp/tiktok-imane-02.mp4"
}

In [2]:
msg.to_dict()

{'file_data': {'mime_type': 'video/mp4',
  'file_uri': 'gs://dmrc-platforms/test/fyp/tiktok-imane-02.mp4'}}

In [7]:

from promptflow.core import Prompty
from promptflow.core._prompty_utils import convert_prompt_template
prompty = Prompty.load("/workspaces/buttermilk/buttermilk/flows/extract/frames.prompty")
template = convert_prompt_template(prompty._template, api="chat", inputs={})
sys = template[0]['content']
sys


'You are a researcher that analyses news articles and identifies how a text is framed. Frames identify a speaker and their affiliation, a problem definition, cause of the problem, attribution of blame, a solution and a solution addressees from each statement in a news article, if they are present.\n\nFORMAT:\nYou must reply only in a fully valid JSON format. Ensure each field is accurately filled and include a rationale that supports your assessment by referencing specific content and principles.\n\nYour response should include these fields:\n```json\n"framing": [List of JSON dictionaries, ensuring that each statement has its own dictionary:\n            {\n                "speaker_name": "<STRING>",\n                "speaker_affiliation": "<STRING>",\n                "solution_adressee": "<STRING>"\n                "statement": "<STRING>",\n                "problem_definition": "<STRING>",\n                "blame_attribution": "<STRING>",\n                "moral_evaluation": "<STRING>

In [9]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting, FinishReason
import vertexai.generative_models as generative_models


def generate():
  vertexai.init(project="dmrc-platforms", location="us-central1")
  model = GenerativeModel(
    "gemini-1.5-pro-001",
    system_instruction=[sys]
  )
  responses = model.generate_content(
      [text1, video1],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
  )

  for response in responses:
    print(response.text, end="")

text1 = 'refer to video'
video1 = Part.from_uri(
    mime_type="video/mp4",
    uri="gs://dmrc-platforms/test/fyp/tiktok-imane-02.mp4")

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE
    ),
]

generate()

```json
{
    "framing": [
        {
            "speaker_name": "daisy_garden4",
            "speaker_affiliation": "TikTok User",
            "solution_adressee": "Audience",
            "statement": "I can't get over how that woman ignored Imane even though she just tried to comfort her.",
            "problem_definition": "The Italian boxer ignored the Algerian boxer's attempt at comforting her.",
            "blame_attribution": "The Italian boxer is to blame for being unsportsmanlike and rude by ignoring the other boxer.",
            "moral_evaluation": "The Italian boxer's actions are worthy of criticism for a lack of sportsmanship.",
            "recommendation": "The audience should also be outraged by the act of poor sportsmanship. "
        }
    ]
}
```

In [ ]:


#         for _, row in df.iterrows():
#             id_vars = {"id": row["id"], "name": variant["name"], "model": model, "timestamp": pd.to_datetime(datetime.datetime.now())}
#             response =
#             response.update(id_vars)
#             response_df = pd.DataFrame(data=[response])
#             results = pd.concat([results, response_df])
#             break
#         break
#     break


# results

